# Machine Learning

## Table of Contents

- [Motivation and Background](#Motivation-and-Background)
- [Data basics](#Data-Basics)
  - [Exercise 1](#Exercise-1)
- [Cleaning and subsetting data](#Understanding-the-Data)
  - [Exercise 2](#Exercise-2)
- [Model Selection and Assessment](#Model-Selection-and-Assessment)
  - [Exercise 3](#Exercise-3)
- [References](#References)


# Motivation and Background

Research in science policy often involves making use of publically available datasets. As such, most of the data we work with is not perfect. Today, we will be diving into the National Institutes for Health (NIH) grant data. NIH provides a myriad of information about its grants which you can access here: http://projectreporter.nih.gov/reporter.cfm. Unfortunately, since the information draws on disparate sources like eRA databases, Medline, PubMed Central, the NIH Intramural Database, and iEdison, it is often not complete. 

In this workbook, we will examine one application of machine learning that deals with predicting missing information. Often in sciece policy research, we are interested in knowing what areas of science are being funded. We often employ diverse techniques to determine that, including text analytics like topic modeling (more on that in the next workbook). However, we can also just simply look at the department of the PI. NIH provides just such a variable in its data. Unfortunately, this variable is often missing. In this workbook, we will walk through the process of imputing values for such a missing categorical variable. 

## Data basics

The data we need for this exercise is available from the 'umetricsgrants' database on the class server. You learnt last week how to obtain data from a MySQL database using python. At this point, you should refer to the data schema provided to you in the first class. In 'nih_project', there is a variable 'ORG_DEPT' that details the department of the PI of the grant. This will be our outcome variable of interest, and for the purpose of this exercise, you are allowed to use any variables in the tables that have the 'nih' prefix as predictors. However, to get us started, I have created an extract of the data and placed it in the 'homework' in a table called 'MachineLearning'. In this workbook, we will be using the pandas package to read in and manipulate data. Pandas provides an alternative to reading data from MySQL that is more efficient since pandas can directly be used for machine learning.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from IPython.core.display import Image
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import metrics

First we create a sql engine. Replace your MySQL credentials in the statement below

In [ ]:
pdb = create_engine("mysql://<username>:<password>@localhost:3306/homework?charset=utf8")

Below we get the data stored in MachineLearning table.

In [ ]:
data = pd.read_sql('Select * from homework.MachineLearning', pdb)

Now, lets look at what the data looks like. The simple command data.head() gives us a sneak peek.

In [ ]:
data.head()

## Understanding the Data

Our data is represented by a DataFrame. You can think of data frames as a giant spreadsheet which you can program. It's a collection of series (or columns) with a common set of commands that make managing data in Python super easy. For example, if you want to look at the last five rows of ORG_DEPT:

In [ ]:
data["ORG_DEPT"].tail(5)

To see how the data is stored internally, we can use the data.dtypes command

In [ ]:
data.dtypes

Lets look at the variables one by one:

* APPLICATION_ID: Unique identifier for each grant
* CFDA_CODE: CFDA contains detailed program descriptions for 2,292 Federal assistance [programs](#https://www.cfda.gov/)
* YEAR: Year in which grant was awarded
* ACTIVITY:A 3-character code identifying the grant, contract, or intramural activity through which a project is supported. Here is a list of activity [codes](#http://grants.nih.gov/grants/funding/ac_search_results.htm)
* ADMINISTERING_IC: Administering Institute or Center - A two-character code to designate the agency, NIH Institute, or Center administering the grant. See [definitions](#http://grants.nih.gov/grants/glossary.htm#I14).
* ARRA_FUNDED: “Y” indicates a project supported by funds appropriated through the American Recovery and Reinvestment Act of 2009.
* ORG_NAME:  The name of the educational institution, research organization, business, or government agency receiving funding for the grant, contract, cooperative agreement, or intramural project.  
* ORG_DEPT:  The departmental affiliation of the contact principal investigator for a project, using a standardized categorization of departments.  Names are available only for medical school departments.
* STUDY_SECTION:  A designator of the legislatively-mandated panel of subject matter experts that reviewed the research grant application for scientific and technical merit.
* TOTAL_COST: Total project funding from all NIH Institute and Centers for a given fiscal year.
* TOPIC_ID: Using text analysis techniques, a topic_id was assigned to each grant. This topic_id is a key for that topic. You can see what the topic contains by looking in the `topiclda_text` table in `umetricsgrants` database.
* ED_INST_TYPE:  Generic name for the grouping of components across an institution who has applied for or receives NIH funding.

You are free to use more predictor variables than the ones listed above from the`nih_project` table in `umetricsgrants` database. A complete description of all variables is available [here](#http://exporter.nih.gov/about.aspx)

### Exercise 1
Pandas provide some great functions for descriptive statistics. Complete the function below to get summary statistics for variables in the dataset, as well as print out a table of top 10 department names. Look into the describe, head, and value_counts functions to achieve that.

In [ ]:
def printDescrriptiveStats(data):
    """
    Parameters
    ----------
    data : A pandas DataFrame
    
    Returns
    -------
    summary : A Pandas dataframe containing count, mean, standard deviation, 
              minimum, maximum, and the 25th, 50th and 75th percentile
    topDepts : A pandas.core.series.Series containing the top 10 departments
               and their frequencies
    """
    ### BEGIN SOLUTION
    
    return data.describe(), data["ORG_DEPT"].value_counts().head(10) 
    ### END SOLUTION

In [ ]:
# lets see what our data looks like
printDescrriptiveStats(data)

## Cleaning and subsetting Data

Looking at the data, it is clear that there are a lot of missing values in ORG_DEPT. Let us do some basic cleaning of the data. For that, we first need to figure out what variables have missing values. Here is a handy function to do that:


In [ ]:
def print_null_freq(df):
    """
    for a given DataFrame, calculates how many values for 
    each variable is null and prints the resulting table to stdout
    """
    df_lng = pd.melt(df)
    null_variables = df_lng.value.isnull()
    return pd.crosstab(df_lng.variable, null_variables)


In [ ]:
# Lets see what our NULL values look like
print_null_freq(data)

Now, that we have a better understanding of the issues our dataset has, lets go ahead and write some code to deal with them.

### Exercise 2


Complete the function below to take in a dataframe, a column name in the dataframe, a value, and return a cleaned dataframe that is subsetted to remove the rows containing the specified value in the specified column. Your function should be equipped to deal with Null values specified in the value parameter. Look into the notnull() function in pandas.

In [ ]:
def cleanData(data, column, value):
    """
    Parameters
    ----------
    data : A pandas DataFrame
    column : Name of the column on the dataframe
    value : The value to be removed from the dataframe in the specified column.
    
    Returns
    -------
    cleanData : A Pandas dataframe containing only rows that did not have the 
                specified value in the specified column
    """
    if(column not in list(data.columns.values)):
        print("ERROR : Column you specified not present in the dataframe")
        return
    
    ### BEGIN SOLUTION
    if value.upper() == "NULL":
        dataClean = data[pd.notnull(data[column])]
    else:
        dataClean = data[data[column] != value]
    return dataClean
    ### END SOLUTION

Clearly, if the department name is missing, we cannot use that data to train our classifier. Fortunately, we have our cleanData function that will take care of the issue.

In [ ]:
dataC = cleanData(data, "ORG_DEPT", "null")
print_null_freq(dataC)

That did clean most of the NULL values for us. There might be other values in ORG_DEPT we might want to clean, so lets print a frequency table for ORG_DEPT

In [ ]:
dataC["ORG_DEPT"].value_counts()

The values of NONE, MISCELLANEOUS and No CODE ASSIGNED are also useless for us. Similarly, we should standardize the department names by converting everything to upper case. In fact, let us go one step further and convert all categorical variables to upper case.

In [ ]:
# Getting rid of NONE and NO CODE ASSIGNED.
dataC = cleanData(dataC, "ORG_DEPT", "NONE")
dataC = cleanData(dataC, "ORG_DEPT", "NO CODE ASSIGNED")
dataC = cleanData(dataC, "ORG_DEPT", "MISCELLANEOUS")

# Converting to upper case
for col in list(data.columns.values):
    if dataC[col].dtype == np.object_:
        dataC[col] = dataC[col].str.upper()

In [ ]:
print_null_freq(dataC)

Since YEAR, STUDY_SECTION and CFDA_CODE are categorical variables, there is no easy way to impute them. So lets do listwise deletion.

In [ ]:
dataC = cleanData(dataC, "CFDA_CODE", "NULL")
dataC = cleanData(dataC, "STUDY_SECTION", "NULL")
dataC = cleanData(dataC, "YEAR", "NULL")

# Lets see if we have any more null frequencies to deal with
print_null_freq(dataC)

We still have 48 missing values for TOTAL_COST. We can get rid of them, but we can also impute them by taking the mean. In this case, it makes more sense to delete these rows since we have a total of 367,691 records and deleting 48 rows will not make much difference.


In [ ]:
dataC = cleanData(dataC, "TOTAL_COST", "NULL")
print_null_freq(dataC)

Lets take another look at our ORG_DEPT variable, and see if we can combine some departments that are very similar.

In [ ]:
dataC["ORG_DEPT"].value_counts()
len(dataC.index)

It certainly looks like a lot of these departments can be combined into broader areas. This would reduce the number of categories to predict and our model will make better predictions.

In [ ]:
dataC.loc[dataC.ORG_DEPT == "PATHOLOGY",['ORG_DEPT']] = "MEDICINE"
dataC.loc[dataC.ORG_DEPT == "INTERNAL MEDICINE/MEDICINE",['ORG_DEPT']] = "MEDICINE"
dataC.loc[dataC.ORG_DEPT == "FAMILY MEDICINE",['ORG_DEPT']] = "MEDICINE"
dataC.loc[dataC.ORG_DEPT == "NEUROSURGERY",['ORG_DEPT']] = "MEDICINE"
dataC.loc[dataC.ORG_DEPT == "PLASTIC SURGERY",['ORG_DEPT']] = "MEDICINE"
dataC.loc[dataC.ORG_DEPT == "OBSTETRICS &GYNECOLOGY",['ORG_DEPT']] = "MEDICINE"
dataC.loc[dataC.ORG_DEPT == "PHARMACOLOGY",['ORG_DEPT']] = "MEDICINE"
dataC.loc[dataC.ORG_DEPT == "ANESTHESIOLOGY",['ORG_DEPT']] = "MEDICINE"
dataC.loc[dataC.ORG_DEPT == "RADIATION-DIAGNOSTIC/ONCOLOGY",['ORG_DEPT']] = "MEDICINE"
dataC.loc[dataC.ORG_DEPT == "DERMATOLOGY",['ORG_DEPT']] = "MEDICINE"
dataC.loc[dataC.ORG_DEPT == "SURGERY",['ORG_DEPT']] = "MEDICINE"
dataC.loc[dataC.ORG_DEPT == "EMERGENCY MEDICINE",['ORG_DEPT']] = "MEDICINE"
dataC.loc[dataC.ORG_DEPT == "DENTISTRY",['ORG_DEPT']] = "MEDICINE"
dataC.loc[dataC.ORG_DEPT == "ORTHOPEDICS",['ORG_DEPT']] = "MEDICINE"
dataC.loc[dataC.ORG_DEPT == "NEUROLOGY",['ORG_DEPT']] = "MEDICINE"
dataC.loc[dataC.ORG_DEPT == "PEDIATRICS",['ORG_DEPT']] = "MEDICINE"
dataC.loc[dataC.ORG_DEPT == "PHYSIOLOGY",['ORG_DEPT']] = "BIOLOGY"
dataC.loc[dataC.ORG_DEPT == "OTOLARYNGOLOGY",['ORG_DEPT']] = "MEDICINE"
dataC.loc[dataC.ORG_DEPT == "UROLOGY",['ORG_DEPT']] = "MEDICINE"

dataC.loc[dataC.ORG_DEPT == "ANATOMY/CELL BIOLOGY",['ORG_DEPT']] = "BIOLOGY"
dataC.loc[dataC.ORG_DEPT == "OPHTHALMOLOGY",['ORG_DEPT']] = "MEDICINE"
dataC.loc[dataC.ORG_DEPT == "ZOOLOGY",['ORG_DEPT']] = "BIOLOGY"
dataC.loc[dataC.ORG_DEPT == "BIOMEDICAL ENGINEERING",['ORG_DEPT']] = "ENGINEERING (ALL TYPES)"
dataC.loc[dataC.ORG_DEPT == "ZOOLOGY",['ORG_DEPT']] = "BIOLOGY"
dataC.loc[dataC.ORG_DEPT == "PHYSICAL MEDICINE &REHAB",['ORG_DEPT']] = "MEDICINE"
dataC.loc[dataC.ORG_DEPT == "PUBLIC HEALTH &PREV MEDICINE",['ORG_DEPT']] = "OTHER HEALTH PROFESSIONS"
dataC.loc[dataC.ORG_DEPT == "NUTRITION",['ORG_DEPT']] = "OTHER CLINICAL SCIENCES"

# We will also get rid of "ADMINISTRATION
dataC = cleanData(dataC, "ORG_DEPT", "ADMINISTRATION")


## Model Selection and Assessment

Now that we have a clean dataset, we can move on to the fun parts!! The python machine learning libraries do not accept categorical variables, so we need to convert all such variables to dummies first. However, pandas makes it super easy! 

But before we do that, lets split our data variables into predictors (features) and predicted, or dependent and independent variables. 

In [ ]:

#Lets go ahead and split into predictors and predicted
features = [x for x in list(dataC.columns.values) if x not in  ["ORG_DEPT"]]
data_x = dataC[features]
data_y = dataC["ORG_DEPT"]

Now, we can easily converty all categorical variables in `data_x` into dummy/binary variables.

In [ ]:
# Python's sckikit algorithms dont work on categorical variables. Fortunately, Pandas provides an easy way out!

data_x = pd.get_dummies(data_x)

If we're building a model, we're going to need a way to know whether or not it's working. Convincing other is oftentimes the most challenging parts of an analysis. Making repeatable, well documented work with clear success metrics makes all the difference.

For our classifier, we're going to use the following build methodology:

In [ ]:
Image(url="https://s3.amazonaws.com/demo-datasets/traintest.png")

Since we have limited number of features, I wont be going into any feature engineering examples. However, here is a good [tutorial](#http://machinelearningmastery.com/discover-feature-engineering-how-to-engineer-features-and-how-to-get-good-at-it/)

Let us now split our dataset into test and training:


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data_x, data_y, test_size=0.25, random_state=0)

Python's `scikit-learn` is a very well known machine library. It is also well documented and maintained. You can learn all about it [here](#http://scikit-learn.org/stable/). We will be using different classifiers from this library for our predictions in this workbook. 

We will start with the simplest `Logistic Regression` model and see how well we do. You can use any number of metrics to judge your models, but we will be using the accuracy score as our measure. 

In [ ]:
# Lets fit the model
model = LogisticRegression()
model.fit(X_train, y_train)
print(model)

When we print the model, we see different parameters we can adjust to make the model. Using cross-validation is a good way to fine-tune the parameters. A good tutorial on cross-validation can be found [here](#http://scikit-learn.org/stable/modules/cross_validation.html).

Now let's use the model we just fit to make predictions on our test dataset, and see what our accuracy score is.

In [ ]:
expected = y_test
predicted = model.predict(X_test)
accuracy_score(expected, predicted)

We get an accuracy score of 46%. This is not a great score, however, it is much better than random guessing, which would have had a chance of 1/18 of succeeding. The other way to guess would be to take the mode, which in this case is MEDICINE with a frequency of 168031, which would give us an accuracy score of  45%. So logistic regression does better than both. Let's see if the other classifiers can do any better.

### Exercise 3
Complete the function below to train different classifiers from the scikit library. Return the accuracy score. Your goal is to come up with a classifier that gives at least 75% accuracy on the test dataset.Feel free to play around with different parameters of the classifier.

In [ ]:
def classifier(X_train, y_train, X_test, y_test):
    """
    Parameters
    ----------
    X_train : A pandas DataFrame of features used for training the classifier
    y_train : A pandas dataframe of y values used for training the classifier
    X_test, y_test : Use these to test the accuracy of your classifier
    
    Returns
    -------
    accuracy score : a float giving the percent of accurate predictions you made
    """
   

    ### BEGIN SOLUTION
    from sklearn.tree import DecisionTreeClassifier
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    expected = y_test
    predicted = model.predict(X_test)
    return accuracy_score(expected, predicted)*100
    ### END SOLUTION.

In [ ]:
classifier(X_train, y_train, X_test, y_test)

## References
* [Scikit-Learn Documentation](#http://scikit-learn.org/stable/)
* [NIH Reporter Documentation](#http://exporter.nih.gov/about.aspx)